In [10]:
from datetime import datetime
from collections import defaultdict
import calendar

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def get_month_range(target_month):
    year, month = map(int, target_month.split("-"))
    start = datetime(year, month, 1)
    end = datetime(year, month, calendar.monthrange(year, month)[1])
    return start, end

def calculate_prorated_amount(rate, qty, start_date, stop_date, month_start, month_end):
    active_start = max(start_date, month_start)
    active_end = min(stop_date, month_end)
    active_days = (active_end - active_start).days + 1
    total_days_in_month = (month_end - month_start).days + 1
    prorated_amount = (rate * qty) * (active_days / total_days_in_month)
    return round(prorated_amount, 2), active_start.strftime("%Y-%m-%d"), active_end.strftime("%Y-%m-%d")

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    month_start, month_end = get_month_range(target_month)
    grouped_items = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        try:
            start_date = parse_date(item["start_date"])
            stop_date = parse_date(item["stop_date"])

            # Skip if outside the month
            if stop_date < month_start or start_date > month_end:
                continue

            qty = int(item["qty"])
            rate = float(item["rate"])

            amount, bill_start, bill_end = calculate_prorated_amount(
                rate, qty, start_date, stop_date, month_start, month_end
            )
            billing_period = f"{bill_start} to {bill_end}"
            key = (item["item_code"], rate, billing_period)

            grouped_items[key]["qty"] += qty
            grouped_items[key]["amount"] += amount

        except Exception as e:
            print(f"Skipping invalid item due to error: {e}")

    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), values in grouped_items.items():
        line_item = {
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": round(values["amount"], 2),
            "billing_period": billing_period
        }
        line_items.append(line_item)
        total_revenue += values["amount"]

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }


In [11]:
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": 10,
        "rate": "1000",
        "amount": "10000",
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": "10",
        "rate": "1080",
        "amount": "10800",
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": "1080",
        "amount": "16200",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": "5000",
        "amount": 35000,
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": "8",
        "rate": "1100",
        "amount": "8800",
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": "3",
        "rate": "5200",
        "amount": "15600",
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": "20000",
        "amount": "20000",
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": "7000",
        "amount": "14000",
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": "7000",
        "amount": 7000,
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
        "idx": 16,
        "item_code": "Breakout Area",
        "qty": 3,
        "rate": "3000",
        "amount": 9000,
        "start_date": "2024-01-01",
        "stop_date": "2024-01-31",
    },
]


In [12]:
result = generate_monthly_bill(item_list, "2024-11")
import pprint
pprint.pprint(result)

{'line_items': [{'amount': 27000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Executive Desk (4*2)',
                 'qty': 25,
                 'rate': 1080.0},
                {'amount': 5000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Executive Desk (4*2)',
                 'qty': 5,
                 'rate': 1000.0},
                {'amount': 25000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Manager Cabin',
                 'qty': 5,
                 'rate': 5000.0},
                {'amount': 10000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Parking (2S)',
                 'qty': 10,
                 'rate': 1000.0},
                {'amount': 0.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Parking (2S)',
                 